# Setup

In [ ]:
!pip install earthengine-api geemap

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [ ]:
!pip install palettable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 4.9 MB/s eta 0:00:00


In [ ]:
import ee
import palettable
import pandas as pd

## Authenticate to Earth Engine Servers

In [ ]:
import ee
ee.Authenticate()

In [ ]:

ee.Initialize(project="gecocolab")

# Analysis

## Examine the ERA5-Land image collection

Reference the image collection for the ERA5-Land hourly dataset

In [ ]:
era5land = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")

Count the number of available records.

In [ ]:
print(
  f'The ERA5 Land Hourly dataset has {era5land.size().getInfo()} records.'
)

The ERA5 Land Hourly dataset has 655103 records.


Find the earliest and latest data in the collection.

In [ ]:
min_date = ee.Date(
    era5land.aggregate_min('system:time_start')
  ).format().getInfo()
max_date = ee.Date(
    era5land.aggregate_max('system:time_end')
  ).format().getInfo()
print(
    f'Timestamps in the ERA5 Land dataset range from {min_date} to {max_date}.'
)

Timestamps in the ERA5 Land dataset range from 1950-01-01T01:00:00 to 2024-09-25T00:00:00.


## Examine a sample record

In [ ]:
# Get a sample record.
sample = era5land.filterDate('2024', '2024-09-18').first()
sample.getInfo()

{'type': 'Image',
 'bands': [{'id': 'dewpoint_temperature_2m',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [3601, 1801],
   'crs': 'EPSG:4326',
   'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]},
  {'id': 'temperature_2m',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [3601, 1801],
   'crs': 'EPSG:4326',
   'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]},
  {'id': 'skin_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [3601, 1801],
   'crs': 'EPSG:4326',
   'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]},
  {'id': 'soil_temperature_level_1',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [3601, 1801],
   'crs': 'EPSG:4326',
   'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]},
  {'id': 'soil_temperature_level_2',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [3601, 1801],
   'crs': 'EPSG:4326',
   

In [ ]:
# Print out the band names.
sample.bandNames().getInfo()

['dewpoint_temperature_2m',
 'temperature_2m',
 'skin_temperature',
 'soil_temperature_level_1',
 'soil_temperature_level_2',
 'soil_temperature_level_3',
 'soil_temperature_level_4',
 'lake_bottom_temperature',
 'lake_ice_depth',
 'lake_ice_temperature',
 'lake_mix_layer_depth',
 'lake_mix_layer_temperature',
 'lake_shape_factor',
 'lake_total_layer_temperature',
 'snow_albedo',
 'snow_cover',
 'snow_density',
 'snow_depth',
 'snow_depth_water_equivalent',
 'snowfall',
 'snowmelt',
 'temperature_of_snow_layer',
 'skin_reservoir_content',
 'volumetric_soil_water_layer_1',
 'volumetric_soil_water_layer_2',
 'volumetric_soil_water_layer_3',
 'volumetric_soil_water_layer_4',
 'forecast_albedo',
 'surface_latent_heat_flux',
 'surface_net_solar_radiation',
 'surface_net_thermal_radiation',
 'surface_sensible_heat_flux',
 'surface_solar_radiation_downwards',
 'surface_thermal_radiation_downwards',
 'evaporation_from_bare_soil',
 'evaporation_from_open_water_surfaces_excluding_oceans',
 'evap

## Image Output

In [ ]:
from IPython.display import Image

aoi = ee.Geometry.Rectangle(6.6283, 35.4933, 18.5203, 47.0855)
#region = ee.Geometry.BBox(119.1687, 21.7799, 122.981, 25.4234)

url = (
    era5land.filterDate('2024-09-16', '2024-09-19')
        .first()
        .getThumbURL({
          'bands': ['total_precipitation_hourly'],
          'min': 0,
          'max': 5e-3,
          'palette': palettable.scientific.sequential.Bilbao_10.hex_colors,
          'region':aoi,
          'dimensions':'500',
          'format':'jpg'
        })
)
display(Image(url=url))

## Video Output

In [ ]:
aoi = ee.Geometry.Rectangle(6.6283, 35.4933, 18.5203, 47.0855)

url = (
    era5land
    .filterDate('2024-09-16', '2024-09-19')
    .getVideoThumbURL({
      'bands': ['total_precipitation_hourly'],
      'min': 0,
      'max': 5e-3,
      'palette': palettable.scientific.sequential.Bilbao_10.hex_colors,
      'region':aoi,
      'dimensions':'500'
    })
)
display(Image(url=url))

## Interactive Map Output

In [ ]:
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Set visualization parameters.
viz_params = {
  'bands': ['total_precipitation'],
  'min': 0,
  'max': 5e-3,
  'palette': palettable.matplotlib.Inferno_10.hex_colors
}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add the elevation model to the map object.
my_map.add_ee_layer(sample, viz_params, 'Total Precipitation')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

# Where to go next...

For additional examples of time series analysis using [pandas](https://pandas.pydata.org/) and [Altair](https://altair-viz.github.io/) visualization libraries, see this notebook tutorial:

https://colab.sandbox.google.com/github/google/earthengine-community/blob/master/tutorials/time-series-visualization-with-altair/index.ipynb